In [1]:
!pip -q install google-adk litellm huggingface_hub llama-cpp-python==0.2.84 python-dotenv


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 MB 9.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 139.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.3/272.3 kB 24.3 MB/s eta 0:00:00


In [3]:
import os
from huggingface_hub import hf_hub_download

# DeepSeek-Coder 6.7B Instruct (quantized GGUF) – good balance of quality/speed
REPO_ID = "TheBloke/DeepSeek-Coder-6.7B-Instruct-GGUF"
FILENAME = "deepseek-coder-6.7b-instruct.Q4_K_M.gguf"

model_path = hf_hub_download(repo_id=REPO_ID, filename=FILENAME)
model_path


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


deepseek-coder-6.7b-instruct.Q4_K_M.gguf:   0%|          | 0.00/4.08G [00:00<?, ?B/s]

'/root/.cache/huggingface/hub/models--TheBloke--DeepSeek-Coder-6.7B-Instruct-GGUF/snapshots/9e221e6b41cb1bf1c5d8f9718e81e3dc781f7557/deepseek-coder-6.7b-instruct.Q4_K_M.gguf'

In [5]:
!pip -q install --upgrade "llama-cpp-python[server]==0.2.84"


In [15]:
# run this once so the next bash cell can see the path
import os
os.environ["MODEL_PATH"] = model_path  # model_path comes from Step 2
print("Using model at:", os.environ["MODEL_PATH"])


Using model at: /root/.cache/huggingface/hub/models--TheBloke--DeepSeek-Coder-6.7B-Instruct-GGUF/snapshots/9e221e6b41cb1bf1c5d8f9718e81e3dc781f7557/deepseek-coder-6.7b-instruct.Q4_K_M.gguf


In [ ]:
import requests, json
print(requests.get("http://127.0.0.1:8000/v1/models", timeout=5).json())


In [16]:
%%bash
# 3.A1 — start server in background and log to /content/server.log
pkill -f "llama_cpp.server.*--port 8000" || true

nohup python -m llama_cpp.server \
  --model "$MODEL_PATH" \
  --host 127.0.0.1 --port 8000 \
  --model_alias deepseek-local \
  --chat_format chatml \
  --n_ctx 4096 \
  > /content/server.log 2>&1 &

sleep 3
tail -n 50 /content/server.log


llm_load_print_meta: f_max_alibi_bias = 0.0e+00
llm_load_print_meta: f_logit_scale    = 0.0e+00
llm_load_print_meta: n_ff             = 11008
llm_load_print_meta: n_expert         = 0
llm_load_print_meta: n_expert_used    = 0
llm_load_print_meta: causal attn      = 1
llm_load_print_meta: pooling type     = 0
llm_load_print_meta: rope type        = 0
llm_load_print_meta: rope scaling     = linear
llm_load_print_meta: freq_base_train  = 100000.0
llm_load_print_meta: freq_scale_train = 0.25
llm_load_print_meta: n_ctx_orig_yarn  = 16384
llm_load_print_meta: rope_finetuned   = unknown
llm_load_print_meta: ssm_d_conv       = 0
llm_load_print_meta: ssm_d_inner      = 0
llm_load_print_meta: ssm_d_state      = 0
llm_load_print_meta: ssm_dt_rank      = 0
llm_load_print_meta: model type       = 7B
llm_load_print_meta: model ftype      = Q4_K - Medium
llm_load_print_meta: model params     = 6.74 B
llm_load_print_meta: model size       = 3.80 GiB (4.84 BPW) 
llm_load_print_meta: general.name     = 

In [17]:
# 3.A2 — wait for server to be ready
import time, requests, subprocess

base = "http://127.0.0.1:8000"
for i in range(120):  # up to ~6 minutes on CPU
    try:
        r = requests.get(f"{base}/v1/models", timeout=2)
        if r.ok:
            print("Server is ready ✅")
            print(r.json())
            break
    except Exception:
        pass
    if i % 5 == 0:
        print(f"...waiting ({i*3}s)")
    time.sleep(3)
else:
    print("Server not ready; last 100 log lines:")
    subprocess.run(["tail","-n","100","/content/server.log"])


Server is ready ✅
{'object': 'list', 'data': [{'id': 'deepseek-local', 'object': 'model', 'owned_by': 'me', 'permissions': []}]}


In [18]:
# 3.A3 — set env for ADK/LiteLLM
import os
os.environ["OPENAI_API_BASE"] = "http://127.0.0.1:8000/v1"
os.environ["OPENAI_API_KEY"]  = "sk-local-anything"  # dummy
print("Env set. Proceed to the ADK agent + FunctionTool cell.")


Env set. Proceed to the ADK agent + FunctionTool cell.


In [29]:
# If you haven't yet:
# !pip -q install google-adk litellm

import asyncio
from dataclasses import dataclass
from typing import Dict, Any

# ADK bits
from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import FunctionTool
from google.genai import types

# Route ADK → your local OpenAI-compatible server via LiteLLM
from google.adk.models.lite_llm import LiteLlm

# ---------- Simple Function Tool ----------
@dataclass
class BudgetInput:
    square_feet: int
    finish_level: str  # "basic" | "standard" | "premium"

def estimate_kitchen_budget(square_feet: int, finish_level: str) -> Dict[str, Any]:
    finish_level = (finish_level or "standard").lower().strip()
    cost_per_sf = {"basic": 150, "standard": 300, "premium": 600}.get(finish_level, 300)
    base = 4000
    estimate = base + square_feet * cost_per_sf
    line_items = [
        {"item": "Design & Permits", "cost": int(0.07 * estimate)},
        {"item": "Cabinetry & Millwork", "cost": int(0.30 * estimate)},
        {"item": "Appliances", "cost": int(0.18 * estimate)},
        {"item": "Counters & Backsplash", "cost": int(0.15 * estimate)},
        {"item": "Plumbing & Electrical", "cost": int(0.12 * estimate)},
        {"item": "Flooring/Paint/Misc.", "cost": int(0.08 * estimate)},
        {"item": "Contingency", "cost": int(0.10 * estimate)},
    ]
    return {
        "status": "success",
        "estimate_usd": int(estimate),
        "finish_level": finish_level,
        "inputs": {"square_feet": square_feet, "finish_level": finish_level},
        "line_items": line_items,
        "notes": "Heuristic only—validate with contractor bids.",
    }

budget_tool = FunctionTool(func=estimate_kitchen_budget)

# ---------- LLM + Agent ----------
# Your server exposed a model with id 'deepseek-local'. LiteLLM uses "openai/<id>".
local_model = LiteLlm(model="openai/deepseek-local")

kitchen_agent = LlmAgent(
    model=local_model,
    name="kitchen_renovation_agent",
    description="Creates a concise kitchen renovation proposal and uses a budget estimator tool.",
    instruction=(
        "You are a renovation consultant. Produce a 1-page proposal with sections:\n"
        "1) Scope overview; 2) Design concept; 3) Materials (bullets);\n"
        "4) Timeline (phases); 5) Budget summary; 6) Disclaimers.\n"
        "Always call the tool `estimate_kitchen_budget(square_feet, finish_level)` to compute the budget.\n"
        "If the tool returns 'status: success', include total and line items.\n"
        "Write clear, structured Markdown. Be practical and specific."
    ),
    tools=[budget_tool],
)

# ---------- Runner (single turn) ----------
APP_NAME = "adk_kitchen_agent_demo"
USER_ID = "student"
SESSION_ID = "session-001"

async def call_agent_once(user_message: str):
    session_service = InMemorySessionService()
    await session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
    runner = Runner(agent=kitchen_agent, app_name=APP_NAME, session_service=session_service)

    content = types.Content(role="user", parts=[types.Part(text=user_message)])
    events = runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content)

    final_text = None

    def extract_text(content_obj):
        try:
            if content_obj and getattr(content_obj, "parts", None):
                for p in content_obj.parts:
                    t = getattr(p, "text", None)
                    if t:
                        return t
        except Exception:
            pass
        return None

    for event in events:
        # ---- try to print tool request/response if present ----
        tool_name = getattr(event, "tool_name", None)
        parameters = getattr(event, "parameters", None)
        if tool_name and parameters is not None:
            print("→ Tool request:", tool_name, parameters)

        # Some versions put tool response into .content; others in .tool_response
        tool_resp = getattr(event, "tool_response", None)
        if tool_resp is not None:
            print("← Tool response (obj):", str(tool_resp)[:180], "...")
        else:
            txt = extract_text(getattr(event, "content", None))
            # Heuristic: if it looks like a JSON/dict from our tool, log a short preview
            if txt and ("estimate_usd" in txt or "line_items" in txt or "status" in txt):
                print("← Tool response (preview):", txt[:180], "...")

        # ---- try to capture the latest meaningful assistant text ----
        txt = extract_text(getattr(event, "content", None))
        if txt:
            final_text = txt

    if final_text:
        print("\n===== Agent Final Response =====\n")
        print(final_text)
        return final_text
    else:
        print("No final response (check logs above).")
        return ""

prompt = (
    "Create a kitchen renovation proposal for a 180 sq ft space, finish level 'standard'. "
    "The client wants an island with seating for 3, durable quartz counters, and to keep the existing layout if possible. "
    "Target budget under $45k if feasible."
)

final_md = await call_agent_once(prompt)   # <- works in Colab




===== Agent Final Response =====

I'm sorry for any confusion, but as a text-based AI model, I don't have access to an actual budget estimator tool or data about kitchen renovation costs. However, I can provide you with an outline based on your description of the project:

1) **Scope Overview**: The scope of the renovation includes a complete redesign of the existing kitchen which consists of a large island with seating for 3 people and durable quartz counters.

2) **Design Concept**: The design concept involves incorporating an open-concept countertop, that complements the existing layout nicely. We will also add a high gloss finish to accentuate the space.

3) **Materials**: 
   - Countertops: Durable quartz counters (4) with an open-concept design for increased versatility and functionality.
   - Accessories: Tall drawers, compact cabinets with handles for easy access to each item.
   - Finish: High gloss finish to make the space appear modern and elegant.

4) **Timeline**: 
   - D

In [30]:
from pathlib import Path
from datetime import datetime

out_dir = Path("/content/agent_outputs")
out_dir.mkdir(parents=True, exist_ok=True)
ts = datetime.now().strftime("%Y%m%d-%H%M%S")
out_path = out_dir / f"proposal_{ts}.md"
out_path.write_text(final_md or "")
print("Saved:", out_path)


Saved: /content/agent_outputs/proposal_20251017-013048.md
